In [150]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima_model import ARIMA

import plotly.graph_objects as go

import numpy as np


In [56]:
def to_int(value):
    try:
        n = value.strip()
        n = n.replace(",",'')
        return int(n)
    except:
        return value
    
def to_pct(value):
    try:
        n = value.replace("%",'')
        return float(n)
    except:
        return value
    
    
def to_date(value):
    return pd.to_datetime(value)

def predictions(column):
    column = column.dropna()
    data = column.values
    model = ARIMA(data, order=(1, 1, 1))
    model_fit = model.fit(disp=False,transparams=False)
    yhat = model_fit.predict(len(data),len(data)+3, typ='levels')
    return yhat

In [57]:
df = pd.read_csv('data/metro.csv')

In [58]:
drop = ['CBSA Code','Year',' Units ']
df = df.drop(drop, axis = 1)

In [157]:
df.Quarter = df.Quarter.apply(lambda x: to_date(x))
df.Properties = df.Properties.apply(lambda x:to_int(x))
df.Units = df.Units.apply(lambda x: to_int(x))
df['Occupancy'] = df['Occupancy'].apply(lambda x: to_pct(x))
df['Annual Rent Growth'] = df['Annual Rent Growth'].apply(lambda x: to_pct(x))
df['Annual Inventory Growth'] = df['Annual Inventory Growth'].apply(lambda x: to_pct(x))
df['Construction vs. Inventory'] = df['Construction vs. Inventory'].apply(lambda x: to_pct(x))
df['Volume'] = df.Units * df.Occupancy
df['Cash'] = df.Volume * df['Annual Rent Growth']

In [158]:
cities = df.CBSA.unique()

In [159]:
cols = df.columns.to_list()
cols.remove('Quarter')
cols.remove('CBSA')

In [160]:
future = ['3Q2019','4Q2019','1Q2020','2Q2020']
future = [to_date(x) for x in future]

In [64]:
for each in cols:
    
    fig = go.Figure()
    
    for city in cities:
        new = df[df.CBSA == city]
        fig.add_trace(go.Scatter(x = new.Quarter, y = new[each],
                            mode = 'lines',
                            name = city))
        
    fig.update_layout(
        title = {
            'text': "{} vs Time".format(each),
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
        },
        xaxis_title = "Time",
        yaxis_title = each,
    )
    fig.show()
    fig.write_image("{} vs Time.png".format(each))

## Enitre Dataset Trend

In [162]:
delta = {}
for city in cities:
    data = []
    new = df[df.CBSA == city]
    for each in cols:
        try:
            single = new[each].dropna()
            X = np.array(single.keys()).reshape(-1, 1)
            y = single.values
            reg = LinearRegression().fit(X, y)
            data.append(float(reg.coef_))
        except:
            data.append(np.nan)
    delta[city] = data

d = pd.DataFrame.from_dict(delta, orient='index')
d.columns = cols
d.sort_values('Volume', ascending=False).head(10)

,Properties,Units,Occupancy,Annual Rent Growth,Annual Inventory Growth,Construction vs. Inventory,Volume,Cash
"Minneapolis, MN",2.317327,256.067088,-0.064344,0.021775,NaN,0.084833,22051.348305,100368.695115
"Dallas, TX",2.148941,247.292925,-0.006012,-0.033847,NaN,0.084655,20891.391329,-27360.157992
"Chicago, IL",1.881613,232.454511,-0.016733,0.005280,NaN,0.038373,19625.402184,70275.910571
"Atlanta, GA",1.436774,158.622864,-0.055947,-0.020721,0.06293,0.229269,12819.597614,7198.677481
"Boston, MA",1.250615,136.216097,-0.059494,-0.043319,NaN,0.042167,11445.680318,-33345.250876
"Houston, TX",1.460731,159.523889,-0.162153,-0.022607,NaN,0.143192,11267.563869,8182.832946
"Phoenix, AZ",1.014149,139.984723,-0.068165,0.022356,NaN,0.148729,10764.909809,85131.418524
"Denver, CO",0.820198,118.197198,-0.005622,-0.020324,NaN,0.082710,10458.449211,12131.262199
"Seattle, WA",0.643336,94.706938,0.044757,0.016926,NaN,-0.077457,9541.422362,67545.230432
"New York, NY",0.666029,107.552324,-0.012297,-0.013579,NaN,0.104351,9367.396398,-17516.541894


## Last Year Trend

In [166]:
delta = {}
for city in cities:
    data = []
    new = df[df.CBSA == city]
    for each in cols:
        try:
            single = new[each].dropna()[-4:]
            X = np.array(single.keys()).reshape(-1, 1)
            y = single.values
            reg = LinearRegression().fit(X, y)
            data.append(float(reg.coef_))
        except:
            data.append(np.nan)
    delta[city] = data

d = pd.DataFrame.from_dict(delta, orient='index')
d.columns = cols
d.sort_values('Volume', ascending=False).head(10)

,Properties,Units,Occupancy,Annual Rent Growth,Annual Inventory Growth,Construction vs. Inventory,Volume,Cash
"Chicago, IL",2.8,435.6,0.12,-0.14,NaN,-0.70,42664.05,-384197.691
"Atlanta, GA",3.9,487.3,-0.39,-0.15,0.44,-1.58,32544.01,-198034.250
"Dallas, TX",2.0,262.5,0.28,0.22,NaN,-0.09,31369.51,651683.878
"Minneapolis, MN",2.6,207.8,0.32,0.33,NaN,-0.02,28191.58,947650.002
"New York, NY",2.1,315.7,-0.06,-0.34,NaN,-0.06,26301.32,-1193209.581
"Phoenix, AZ",2.9,394.6,-0.28,0.08,NaN,-0.79,25711.13,294505.965
"Houston, TX",1.0,158.7,0.30,-0.65,NaN,0.81,18822.57,-984182.955
"Denver, CO",0.7,106.0,0.60,0.19,NaN,-0.18,18634.19,310739.119
"Cleveland, OH",1.3,133.5,0.64,0.24,NaN,-0.39,17952.68,281570.606
"Philadelphia, PA",1.0,130.2,0.08,-0.23,NaN,0.08,14925.58,-773351.682


In [164]:
d.sort_values('Annual Rent Growth', ascending=False).head(10)

,Properties,Units,Occupancy,Annual Rent Growth,Annual Inventory Growth,Construction vs. Inventory,Volume,Cash
"Kansas City, MO",5.000000e-01,83.9,0.36,1.32,NaN,-0.94,12332.55,1668081.198
"San Jose, CA",-4.000000e-01,-80.8,0.28,0.96,NaN,0.04,-5966.15,538433.552
"Las Vegas, NV",3.000000e-01,38.7,0.65,0.68,NaN,-0.93,5932.64,264628.848
"Minneapolis, MN",2.600000e+00,207.8,0.32,0.33,NaN,-0.02,28191.58,947650.002
"Cleveland, OH",1.300000e+00,133.5,0.64,0.24,NaN,-0.39,17952.68,281570.606
"Dallas, TX",2.000000e+00,262.5,0.28,0.22,NaN,-0.09,31369.51,651683.878
"Denver, CO",7.000000e-01,106.0,0.60,0.19,NaN,-0.18,18634.19,310739.119
"Tampa, FL",7.000000e-01,58.9,0.38,0.09,NaN,0.32,13295.96,201895.126
"Phoenix, AZ",2.900000e+00,394.6,-0.28,0.08,NaN,-0.79,25711.13,294505.965
"Pittsburgh, PA",2.482534e-17,0.7,-0.25,0.05,NaN,0.77,-3376.19,50646.497
